## 마스크드 언어 모델과 토크나이저
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다. 예를 들어서 A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없습니다. A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문입니다.

In [3]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

In [5]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [6]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [7]:
# 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있습니다.
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [8]:
# 토크나이저로 변환된 결과에서 attention_mask를 통해서 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크를 확인할 수 있습니
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


## [MASK] 토큰 예측하기

In [9]:
#FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줍니다. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.

from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [10]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.762112021446228,
  'sequence': 'soccer is a really fun sport.',
  'token': 4368,
  'token_str': 'sport'},
 {'score': 0.2034197747707367,
  'sequence': 'soccer is a really fun game.',
  'token': 2208,
  'token_str': 'game'},
 {'score': 0.012208552099764347,
  'sequence': 'soccer is a really fun thing.',
  'token': 2518,
  'token_str': 'thing'},
 {'score': 0.001863026525825262,
  'sequence': 'soccer is a really fun activity.',
  'token': 4023,
  'token_str': 'activity'},
 {'score': 0.0013354825787246227,
  'sequence': 'soccer is a really fun field.',
  'token': 2492,
  'token_str': 'field'}]